In [ ]:
import pygmsh
import meshio
from fenics import *

In [ ]:
N = 80
brain_radius = 0.1
fluid_radius = brain_radius*1.2
ventricle_radius = 0.03
aqueduct_width = 0.005
canal_width = 0.02
canal_length = 0.2

# subdomain ids
fluid_id = 2
porous_id = 1

# boundary ids
interface_id = 1
rigid_skull_id = 2
spinal_outlet_id = 3


h = 1.0/N
geom = pygmsh.opencascade.Geometry(
  characteristic_length_min=h,
  characteristic_length_max=h,
  )

brain = geom.add_ball((0,0,0), brain_radius)
ventricle = geom.add_ball((0,0,0), ventricle_radius)
aqueduct = geom.add_cylinder((0,0,0), (0,0,-brain_radius), aqueduct_width)
brain = geom.boolean_difference([brain], [ventricle, aqueduct])

spinal_canal = geom.add_cylinder((0,0,0), (0,0,-canal_length), canal_width)
fluid = geom.add_ball((0,0,0), fluid_radius)
fluid = geom.boolean_union([fluid, spinal_canal])
#fluid = geom.boolean_difference([fluid], [brain])
tot = geom.boolean_fragments([fluid], [brain])

geom.add_physical(fluid, fluid_id)
geom.add_physical(brain, porous_id)

mesh = pygmsh.generate_mesh(geom)


In [ ]:
meshio.write(f"../meshes/ideal_brain_subd_3D_N{N}.xdmf",
             meshio.Mesh(points=mesh.points, cells={"tetra": mesh.cells_dict["tetra"]},
                                   cell_data={"subdomains":mesh.cell_data["gmsh:physical"]}))


In [ ]:

infile = XDMFFile(f"../meshes/ideal_brain_subd_3D_N{N}.xdmf")
mesh = Mesh()
infile.read(mesh)
mf = MeshFunction("size_t", mesh, 3, 0)
infile.read(mf, "subdomains")
boundaries = MeshFunction("size_t", mesh, 2, 0)

# set internal interface
for f in facets(mesh):
    domains = []
    for c in cells(f):
        domains.append(mf[c])

    domains = list(set(domains))
    if len(domains) > 1:
        boundaries[f] = interface_id
        
# set rigid skull boundary
rigid_skull = CompiledSubDomain("on_boundary",
                                canal_length=canal_length)

rigid_skull.mark(boundaries, rigid_skull_id)

# set spinal outlet 
spinal_outlet = CompiledSubDomain("near(x[2], - canal_length)",
                                canal_length=canal_length)
spinal_outlet.mark(boundaries, spinal_outlet_id)

boundaries_outfile = XDMFFile(f"../meshes/ideal_brain_boundaries3D_N{N}.xdmf")
#subdomains_outfile = XDMFFile(f"../meshes/ideal_brain_subdomains3D_N{N}.xdmf")

boundaries_outfile.write(boundaries)
#subdomains_outfile.write(mf)
boundaries_outfile.close()
#subdomains_outfile.close()